# GRASS GIS LiDAR Derived Hydrography Flowlines

This notebook prototypes development of LiDAR elevation derived flow path data in [GRASS GIS]( https://grass.osgeo.org).

## Install GRASS GIS

Run this notebook within the GRASS GIS environment.  If GRASS is not currently installed download from: [https://grass.osgeo.org/download/](https://grass.osgeo.org/download/).

## Checking the python environment

In [ ]:
import sys
print (sys.version)

## Checking the file path

In [ ]:
%%bash
pwd

In [ ]:
!pwd

## Create a new GRASS GIS Project

Creating a new [GRASS GIS]( https://grass.osgeo.org) project.  For LiDAR derived hydrography the project will be conducted in the same Coordinate Reference System (CRS) of the LiDAR based digitial elevation model (DEM) downloaded in the orginal product resolution (OPR) folder.  Flow path analysis may be developed from pre-existing DEMs or DEMs generated from point cloud data.

The `ls` command provides a directory listing at the present working directory (pwd). 

In [ ]:
!ls 

Next, double check the CRS of the DEM data to ensure consistent setup of the GRASS project using the same (CRS).

In [ ]:
!ls ./OPR/*.tif

Selecting one of the GeoTIFF files, read the coordinate system with [gdalinfo](https://gdal.org/programs/gdalinfo.html#gdalinfo).

In [ ]:
%%bash
gdalinfo ./OPR/D03_37_10875003_20160228.tif

Based on the above output, the command below will create (with the -`c` flag) a GRASS GIS project in North Carolina State Plane projection with US Survey Foot (USFT) units using the corresponding code: [EPSG:6543](https://epsg.org/crs_6543/NAD83-2011-North-Carolina-ftUS.html).  The following GRASS GIS command builds the empty GRASS project at the local home directory path (and exits with the -`e` flag).

In [ ]:
%%bash
grass -c EPSG:6543 -e ~/grassdata/GRASS_6543/

Information about running GRASS GIS in jupyter notbooks is available at the following [link](https://grass.osgeo.org/grass83/manuals/libpython/index.html).  The next cell will initiate the GRASS GIS project just created and import requried libraries.

In [ ]:
# Load Python environment and IPython package.
import subprocess
import sys

# Query GRASS GIS as to Python packages.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Load the GRASS GIS packages.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS GIS project previously created above.
session = gj.init("~/grassdata", "GRASS_6543", "PERMANENT")

Check the current GRASS GIS environment and list the mapset, database, location and projection information.

In [ ]:
%%bash
g.gisenv
g.proj -p

## Import / Create LiDAR DEM data.

Data (DEM / LiDAR) for this exercise is available at the NOAA Digital Coast bulk download site:  

- DEM:  [https://chs.coast.noaa.gov/htdata/raster2/elevation/NorthCarolina_DEM_2015P3_6205/](https://chs.coast.noaa.gov/htdata/raster2/elevation/NorthCarolina_DEM_2015P3_6205/)
- LiDAR:  [https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/6209/index.html](https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/6209/index.html)

Import the DEM with GRASS GIS [r.import](https://grass.osgeo.org/grass83/manuals/r.import.html).

In [ ]:
!r.import in=Reference/NC_P3_2015_TEM_Coleridge_SE_opr.tif out=NC_P3_2015_TEM_Coleridge_SE_opr --overwrite

In [ ]:
!g.list rast

In [ ]:
!g.region -p rast=NC_P3_2015_TEM_Coleridge_SE_opr

In [ ]:
# Define Grass Jupyter Map 
NC_P3_2015_TEM_opr_map = gj.Map()
# Add raster data and legend to the map
NC_P3_2015_TEM_opr_map.d_rast(map="NC_P3_2015_TEM_Coleridge_SE_opr")
NC_P3_2015_TEM_opr_map.d_legend(raster="NC_P3_2015_TEM_Coleridge_SE_opr", color="red")
# Display the map
NC_P3_2015_TEM_opr_map.show()

On USGS QL2-based projects BHI traditional data processing mode is to conduct analysis at the USGS 7.5 minute quadrangle series level.  In this exercise, we'll utilize the 3.75 minute quarter quadrangle series as the atomic unit for proceessing and import a vector cell map for this TEM area of interest (AOI).

In [ ]:
!v.import in=Reference/cellgrid_3_75minute_tem.gpkg out=cellgrid_3_75minute_tem

In [ ]:
# Define Grass Jupyter Map 
NC_P3_2015_TEM_opr_map = gj.Map()
# Add raster data and legend to the map
NC_P3_2015_TEM_opr_map.d_rast(map="NC_P3_2015_TEM_Coleridge_SE_opr")
NC_P3_2015_TEM_opr_map.d_legend(raster="NC_P3_2015_TEM_Coleridge_SE_opr", color="red")
NC_P3_2015_TEM_opr_map.d_vect(map="cellgrid_3_75minute_tem", fill="none", attribute_column="CELL_NAME", xref="center", label_size=12)
# Display the map
NC_P3_2015_TEM_opr_map.show()

List the cellgrid vector table columns with [db.columns](https://grass.osgeo.org/grass84/manuals/db.columns.html) and [v.db.select](https://grass.osgeo.org/grass84/manuals/v.db.select.html).

In [ ]:
!db.columns table=cellgrid_3_75minute_tem

In [ ]:
!v.db.select map=cellgrid_3_75minute_tem column=CELL_NAME | tail -n +2 | sed 's/ /_/g'

Output the above command as a file list.

In [ ]:
!v.db.select map=cellgrid_3_75minute_tem column=CELL_NAME | tail -n +2 | sed 's/ /_/g' > Reference/CellGrid_3_75Minute_TEM.txt

In [ ]:
!cat Reference/CellGrid_3_75Minute_TEM.txt

Extract 3.75 minute quadrangle units for processing with a shell script.

In [ ]:
!sh ./make_quad_extract.sh

In [ ]:
!g.list vect

Individual steps follow in the next cells to demonstrate processing using the Coleridge SE 3.75 minute unit, starting with setting the region with [g.region](https://grass.osgeo.org/grass84/manuals/g.region.html).

In [ ]:
!g.region -pa res=3.125 vect=Coleridge_SE_Mask

In traditional BHI flow line processing, the region has normally been extended (by 3600 feet) to ensure consistent map boundary edge matching.  For this exercise, given limited DEM coverage, unit boundary analysis regions will be extended by 600 feet at the native resolution of the DEM data -- 3.125 feet.  Traditional processsing for QL2 has normally operated at 2 foot pixel resoultion and for QL1 at 1 foot pixel resolution.

In [ ]:
!g.region -p -a n=n+600 s=s-600 w=w-600 e=e+600

Geomorphic landform identification with [r.geomorphon](https://grass.osgeo.org/grass84/manuals/r.geomorphon.html) can be instrumental in culvert detection when hydro-enforcement is required. This next cell takes about ~6 minutes to run.

In [ ]:
%%bash
START=$(date +%s);
sleep 1; 
echo $START
r.geomorphon elevation=NC_P3_2015_TEM_Coleridge_SE_opr forms=Coleridge_SE_geomorph search=40 skip=5 flat=6 dist=6 --overwrite
END=$(date +%s);
echo ----- $((END-START)) seconds -----

Optionally, mode filtering of the geomorphic landform data with [r.neighbors](https://grass.osgeo.org/grass84/manuals/r.neighbors.html) can help with interpretation depending on the dataset.

In [ ]:
!r.neighbors in=Coleridge_SE_geomorph out=Coleridge_SE_geomorph_mode3 method=mode nprocs=6 --overwrite

In [ ]:
!r.watershed --overwrite elevation=NC_P3_2015_TEM_Coleridge_SE_opr accumulation=Coleridge_SE_ACC

In the next cell the computational region is reduced by 100 feet. 

In [ ]:
!g.region -p -a n=n-100 s=s+100 w=w+100 e=e-100

In [ ]:
!r.out.gdal -f --overwrite input=Coleridge_SE_ACC output=Reference/Coleridge_SE_ACC.tif format=GTiff type=Float64 createopt=COMPRESS=LZW,PREDICTOR=3,TILED=YES,BLOCKXSIZE=128,BLOCKYSIZE=128,BIGTIFF=YES overviews=5

In [ ]:
!r.out.gdal -f --overwrite input=Coleridge_SE_geomorph output=Reference/Coleridge_SE_geomorph.tif format=GTiff type=Byte createopt=COMPRESS=LZW,PREDICTOR=2,TILED=YES,BLOCKXSIZE=128,BLOCKYSIZE=128,BIGTIFF=YES overviews=5

In [ ]:
!r.stream.extract elevation="NC_P3_2015_TEM_Coleridge_SE_opr" accumulation=Coleridge_SE_ACC direction=Coleridge_SE_Direction threshold=1000 stream_rast=Coleridge_SE_Stream --overwrite

The next cells add stream ordering with the GRASS GIS Add-on [r.stream.order](https://grass.osgeo.org/grass84/manuals/addons/r.stream.order.html).

Install the Add-on.

In [ ]:
!r.stream.order

In [ ]:
# !g.extension extension=r.stream.order

In [ ]:
!r.stream.order --overwrite stream_rast=Coleridge_SE_Stream direction=Coleridge_SE_Direction elevation=NC_P3_2015_TEM_Coleridge_SE_opr accumulation=Coleridge_SE_ACC stream_vect=Coleridge_SE_Stream strahler=Coleridge_SE_Strahler horton=Coleridge_SE_Horton shreve=Coleridge_SE_Shreve hack=Coleridge_SE_Hack topo=Coleridge_SE_Topo

Generalization with [v.generalize](https://grass.osgeo.org/grass84/manuals/v.generalize.html) using the GRASS GIS snakes algorithm.

In [ ]:
!v.generalize --overwrite input=Coleridge_SE_Stream output=Coleridge_SE_Order_Smooth method=snakes threshold=2

In [ ]:
!v.out.ogr --overwrite input=Coleridge_SE_Order_Smooth output=Reference/Coleridge_SE_Order_Smooth.sqlite format=SQLite